# Converting to classification problem 

Generated Bad links from graph which are not in graph and whose shortest path is greater than 2. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd
import datetime 
import time 
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams
from sklearn.cluster import MiniBatchKMeans, KMeans
import math
import pickle
import os
#!pip install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
import random as rnd


from sklearn.model_selection import train_test_split

In [2]:
connections=9437519
nodes=1862220

In [3]:
%%time
if not os.path.isfile('Bad_connections.p'):
    r = csv.reader(open('train.csv','r'))
    edges = dict()
    for edge in r:
        edges[(edge[0], edge[1])] = 1
        
        
    bad_edges_set = set([])
    while (len(bad_edges_set)< connections):
        s_node=rnd.randint(1,nodes)
        d_node=rnd.randint(1,nodes)
        tmp = edges.get((s_node,d_node),0)
        if tmp == 0 and s_node!=d_node:
            try:
                if nx.shortest_path_length(g,source=s_node,target=d_node) > 2: 

                    bad_edges_set.add((s_node,d_node))
                else:
                    continue  
            except:  
                    bad_edges_set.add((s_node,d_node))              
        else:
            continue
    pickle.dump(bad_edges_set,open('Bad_connections.p','wb'))
else:
    bad_edges_set = pickle.load(open('Bad_connections.p','rb'))

Wall time: 44.5 s


In [4]:
bad_edges_set = pickle.load(open('Bad_connections.p','rb'))
len(bad_edges_set)

9437519

## Splitting positive and negative connections

In [5]:
if (not os.path.isfile('Good_edges.csv')):
    Good_edges = pd.read_csv('train.csv',usecols=[0,1], names=['source', 'destination'], header=None)
    Bad_edges = pd.DataFrame(list(bad_edges_set), columns=['source', 'destination'])

    y_train_pos=np.ones(len(Good_edges))
    y_train_neg=np.zeros(len(Bad_edges))
    
    print("Good edges in graph = ", Good_edges.shape[0])
    print("Bad edges in graph = ", Bad_edges.shape[0])

    #saving
    Bad_edges.to_csv('Bad_edges.csv',header=False, index=False)
else:
    #Graph from Traing data only 
    print('deleting ........')
    del bad_edges_set

Good edges in graph =  9437519
Bad edges in graph =  9437519


In [6]:
#final train and test data sets
if (not os.path.isfile('Classification.csv')):
    
    Good_edges = pd.read_csv('train.csv',usecols=[0,1], names=['source', 'destination'], header=None)
    Bad_edges = pd.read_csv('Bad_edges.csv', names=['source', 'destination'])


    X_train = Good_edges.append(Bad_edges,ignore_index=True)
    y_train = np.concatenate((y_train_pos,y_train_neg))

    
    X_train.to_csv('Classification.csv',header=False,index=False)
    pd.DataFrame(y_train.astype(int)).to_csv('train_y.csv',header=False,index=False)